In [1]:
import os
GPU_id = 0
os.environ['CUDA_VISIBLE_DEVICES'] = str(GPU_id)

In [2]:
import warnings
warnings.filterwarnings("ignore")
import math
import pandas as pd
import numpy as np
import time
import tensorflow as tf
from rnn import PlasticcRNN
import matplotlib.pyplot as plt
%matplotlib inline

print(tf.__version__)

1.11.0-dev20180910


In [3]:
params = {
        'load_path':'weight/rnn.npy',
        'hidden':64,
        'bottleneck':True,
        'classes':14,
        'num_features':4,
        'embedding_size':4,
        'stratified':True,
        'objective':'multiclassification',
        'metric':'cross_entropy',
        'save_path':'weights',      
        'epochs':100,
        'early_stopping_epochs':10,
        'learning_rate':0.01,
        'batch_size':2048,
        'verbosity':10,
        #'folds':4
    }

In [4]:
model = PlasticcRNN(**params)

In [5]:
%%time
train = pd.read_pickle('train_rnn.pkl')
test = pd.read_pickle('test_rnn.pkl')

CPU times: user 2.9 s, sys: 13.6 s, total: 16.5 s
Wall time: 16.5 s


In [6]:
%%time
train_bn = model.predict_bottleneck(train)

restore RNN/rnn3/bidirectional_rnn/fw/gru_cell/gates/kernel:0
restore RNN/rnn3/bidirectional_rnn/fw/gru_cell/gates/bias:0
restore RNN/rnn3/bidirectional_rnn/fw/gru_cell/candidate/kernel:0
restore RNN/rnn3/bidirectional_rnn/fw/gru_cell/candidate/bias:0
restore RNN/rnn3/bidirectional_rnn/bw/gru_cell/gates/kernel:0
restore RNN/rnn3/bidirectional_rnn/bw/gru_cell/gates/bias:0
restore RNN/rnn3/bidirectional_rnn/bw/gru_cell/candidate/kernel:0
restore RNN/rnn3/bidirectional_rnn/bw/gru_cell/candidate/bias:0
restore RNN/rnn5/bidirectional_rnn/fw/output_projection_wrapper/gru_cell/gates/kernel:0
restore RNN/rnn5/bidirectional_rnn/fw/output_projection_wrapper/gru_cell/gates/bias:0
restore RNN/rnn5/bidirectional_rnn/fw/output_projection_wrapper/gru_cell/candidate/kernel:0
restore RNN/rnn5/bidirectional_rnn/fw/output_projection_wrapper/gru_cell/candidate/bias:0
restore RNN/rnn5/bidirectional_rnn/fw/output_projection_wrapper/kernel:0
restore RNN/rnn5/bidirectional_rnn/fw/output_projection_wrapper/bia

  0%|          | 0/3 [00:00<?, ?it/s]

restore RNN/rnn4/bidirectional_rnn/bw/gru_cell/gates/bias:0
restore RNN/rnn4/bidirectional_rnn/bw/gru_cell/candidate/kernel:0
restore RNN/rnn4/bidirectional_rnn/bw/gru_cell/candidate/bias:0


4it [00:03,  1.20it/s]                       

CPU times: user 1min 28s, sys: 8.07 s, total: 1min 36s
Wall time: 11.5 s


In [7]:
%%time
test_bn = model.predict_bottleneck(test)

restore RNN/rnn3/bidirectional_rnn/fw/gru_cell/gates/kernel:0
restore RNN/rnn3/bidirectional_rnn/fw/gru_cell/gates/bias:0
restore RNN/rnn3/bidirectional_rnn/fw/gru_cell/candidate/kernel:0
restore RNN/rnn3/bidirectional_rnn/fw/gru_cell/candidate/bias:0
restore RNN/rnn3/bidirectional_rnn/bw/gru_cell/gates/kernel:0
restore RNN/rnn3/bidirectional_rnn/bw/gru_cell/gates/bias:0
restore RNN/rnn3/bidirectional_rnn/bw/gru_cell/candidate/kernel:0
restore RNN/rnn3/bidirectional_rnn/bw/gru_cell/candidate/bias:0
restore RNN/rnn5/bidirectional_rnn/fw/output_projection_wrapper/gru_cell/gates/kernel:0
restore RNN/rnn5/bidirectional_rnn/fw/output_projection_wrapper/gru_cell/gates/bias:0
restore RNN/rnn5/bidirectional_rnn/fw/output_projection_wrapper/gru_cell/candidate/kernel:0
restore RNN/rnn5/bidirectional_rnn/fw/output_projection_wrapper/gru_cell/candidate/bias:0
restore RNN/rnn5/bidirectional_rnn/fw/output_projection_wrapper/kernel:0
restore RNN/rnn5/bidirectional_rnn/fw/output_projection_wrapper/bia

1237it [10:09,  2.03it/s]                          

CPU times: user 4h 37min 31s, sys: 19min 42s, total: 4h 57min 14s
Wall time: 10min 14s


In [8]:
print(train_bn.shape,train.object_id.unique().shape)
print(test_bn.shape,test.object_id.unique().shape)

(7848, 16) (7848,)
(2531953, 16) (2531953,)


In [9]:
%%time
train_bn = pd.DataFrame(train_bn,columns=['bottleneck%d'%i for i in range(train_bn.shape[1])])
train_bn['object_id'] = train.object_id.unique()

CPU times: user 540 ms, sys: 32 ms, total: 572 ms
Wall time: 14.3 ms


In [10]:
train_bn.head()

,bottleneck0,bottleneck1,bottleneck2,bottleneck3,bottleneck4,bottleneck5,bottleneck6,bottleneck7,bottleneck8,bottleneck9,bottleneck10,bottleneck11,bottleneck12,bottleneck13,bottleneck14,bottleneck15,object_id
0,39.732155,0.481644,9.545365,0.620393,0.356217,36.236855,0.001318,0.045979,0.075107,7.429890,0.131433,0.000731,0.459891,56.466618,0.147016,20.742085,615
1,2.141680,6.474416,8.759016,0.114259,2.240717,9.736773,7.122033,10.171884,3.942685,1.042941,1.473609,0.025092,6.322167,1.655637,3.261945,18.709692,713
2,1.139722,28.513433,0.475543,0.954773,9.411211,1.062097,18.426241,2.884041,2.424165,1.874315,0.666815,10.709194,5.510746,0.388815,8.288032,2.827859,730
3,13.054772,24.500605,3.417212,2.564083,0.949257,1.549924,21.927139,6.542566,6.207687,4.959152,2.384894,2.566334,9.735006,0.954084,17.784241,3.097169,745
4,2.804562,22.519880,5.931814,2.715422,2.990605,2.720283,20.145359,4.909352,5.238781,2.418497,0.306975,3.099151,7.483632,0.263369,8.544587,1.836023,1124


In [11]:
test_bn = pd.DataFrame(test_bn,columns=['bottleneck%d'%i for i in range(test_bn.shape[1])])
test_bn['object_id'] = test.object_id.unique()

In [12]:
test_bn.head()

,bottleneck0,bottleneck1,bottleneck2,bottleneck3,bottleneck4,bottleneck5,bottleneck6,bottleneck7,bottleneck8,bottleneck9,bottleneck10,bottleneck11,bottleneck12,bottleneck13,bottleneck14,bottleneck15,object_id
0,5.194526,0.741064,6.832206,2.447083,2.201819,3.955753,3.760406,8.530709,9.972454,0.191506,4.992706,1.357791,2.810582,0.495058,4.425712,11.054269,9848799
1,0.697726,8.406124,1.677688,2.992078,7.273104,0.361961,9.302008,2.370127,8.173873,0.674359,2.505515,3.722713,4.316256,0.836422,5.697777,4.040637,9848847
2,0.805780,6.492369,2.065151,2.698128,3.258145,0.945475,3.464690,3.637921,9.949050,1.462446,8.333170,5.366208,1.282938,0.235688,3.341774,7.788818,9848868
3,0.589768,1.544212,4.751875,1.506920,4.491727,1.666431,10.164309,4.049687,6.018585,0.429728,0.928146,4.202436,4.040009,0.154402,2.161732,12.156486,9848939
4,1.401312,9.299484,2.887816,0.746563,4.639925,0.638851,5.328112,2.093352,5.907254,1.124667,3.822571,6.906983,3.224484,1.971383,7.659608,5.114751,9848960


In [13]:
%%time
train_bn.to_pickle('train_bn.pkl')
test_bn.to_pickle('test_bn.pkl')

CPU times: user 92 ms, sys: 328 ms, total: 420 ms
Wall time: 417 ms
